# suggest_agent_tools.py 테스트 노트북

이 노트북은 `suggest_agent_tools.py`에 정의된 `PolicySearchTool`과 `PolicyDetailTool`을 사용하여 `source/vectorstore`에 저장된 정책을 검색하고, 선택된 정책의 상세(필요 서류 등)를 JSON 형식으로 출력합니다.

실행 환경: 노트북의 각 셀을 순서대로 실행하세요.

In [11]:
# 1) 필요한 모듈 임포트 및 도구 불러오기
from suggest_agent_tools import PolicySearchTool, PolicyDetailTool
import json

# (참고) suggest_agent_tools.py 내부에서 이미 load_dotenv()를 호출하므로 별도 로드 불필요

# 인스턴스 생성 (header_db와 body_db의 기본 경로를 사용)
search_tool = PolicySearchTool()
detail_tool = PolicyDetailTool()

print("PolicySearchTool, PolicyDetailTool 인스턴스 생성 완료")

PolicySearchTool, PolicyDetailTool 인스턴스 생성 완료


In [9]:
# (중복된 검색 셀이 있을 수 있어 동일한 로직으로 업데이트합니다.)
# 2) 정책 검색 예시 (사용자 질문을 변경해 테스트하세요)
query = "서울시에 사는 27세 청년입니다. 보증금이 부족한데 지원을 받고싶어요 찾아주세요."
print("질문:", query)

# 상위 3개 후보를 받아와서 raw contents를 출력해 원본 데이터를 확인합니다.
try:
    retriever = search_tool.header_vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3}
    )
    headers = retriever.get_relevant_documents(query)
except Exception as e:
    print("헤더 검색 실패:", e)
    headers = []

print("\n[Top-3 검색 결과 - raw header page_content]")
for i, h in enumerate(headers, start=1):
    print(f"\n--- 후보 {i} ---")
    try:
        parsed = json.loads(h.page_content)
        print(json.dumps(parsed, ensure_ascii=False, indent=2))
    except Exception:
        print(h.page_content)

# 간단한 키워드 기반 재랭킹: '보증금','이자' 관련 용어를 포함한 후보 우선 선택
keywords = ["보증금", "이자", "중개보수", "월세지원", "이사비"]
best = None
best_score = 0
for h in headers:
    text = (h.page_content or "").lower()
    score = sum(text.count(k) for k in keywords)
    if score > best_score:
        best_score = score
        best = h

if best is None and headers:
    best = headers[0]

if not best:
    print("\n검색된 정책이 없습니다.")
    policy = {"status": "error", "message": "관련 정책을 찾을 수 없습니다."}
else:
    try:
        header_content = json.loads(best.page_content)
    except Exception:
        header_content = {"policy_name": None, "original_collection_name": None, "summary": None}

    summary = header_content.get("summary")
    if not summary:
        for alt in ("description", "short_summary", "summary_text", "policy_summary"):
            if header_content.get(alt):
                summary = header_content.get(alt)
                break

    print("\n[선택된 정책]")
    print("정책명:", header_content.get("policy_name"))
    collection_name = header_content.get("original_collection_name")
    print("collection_name:", collection_name)
    print("요약:", summary if summary else "정보 없음")

    policy = {
        "status": "success",
        "policy_name": header_content.get("policy_name"),
        "collection_name": collection_name,
        "summary": summary
    }


질문: 서울시에 사는 27세 청년입니다. 보증금이 부족한데 지원을 받고싶어요 찾아주세요.

[Top-3 검색 결과 - raw header page_content]

--- 후보 1 ---
{
  "summary": {
    "policy_category": "주거",
    "policy_name": "2025년 서울시 청년월세지원 사업",
    "policy_summary": "서울시에 거주하는 19세에서 39세 이하 청년을 대상으로 월세 부담을 줄여주기 위해 최대 월 20만원, 연 최대 240만원까지 지원하는 사업입니다. 임대차계약서에 명시된 월세만 지원하며, 보증금과 월세를 합산한 금액이 일정 기준 이하인 경우 신청할 수 있습니다. 소득 기준은 가구당 중위소득 150% 이하이며, 신청은 온라인으로 접수합니다. 선정자는 신한 청년드림통장 개설 후 지원금을 받게 됩니다.",
    "policy_target": "서울시에 거주하는 19~39세 이하 무주택 청년 가구",
    "policy_benefit": "월 최대 20만원, 최대 12개월간 총 240만원 지원"
  },
  "policy_name": "서울시 청년월세지원정책",
  "original_collection_name": "seoulsiceongnyeonweolsejiweonjeongcaeg_body"
}

--- 후보 2 ---
{
  "summary": {
    "policy_category": "주거",
    "policy_name": "2025년 청년 부동산 중개보수 및 이사비 지원사업",
    "policy_summary": "서울시에 새로 전입하거나 이사한 만 19세에서 39세 청년가구를 대상으로 부동산 중개수수료와 이사비용을 최대 40만원까지 실비로 지원하는 사업입니다. 소득과 재산 기준을 충족하는 무주택 청년이 신청할 수 있으며, 사회적 약자와 주거취약 청년을 우선 선발합니다.",
    "policy_target": "만 19세~39세 서울시 내 전입 또는 이사 완료한 

In [10]:
# 선택된 정책의 상세 정보 추출
if policy.get("status") == "success":
    collection_name = policy.get("collection_name")
    print("\n[선택된 정책의 상세 정보 추출]")
    print("Collection Name:", collection_name)
    
    # PolicyDetailTool을 사용하여 상세 정보 추출
    details = detail_tool.run(collection_name)
    
    if details.get("status") == "success":
        detail_info = details.get("details", {})
        
        # 결과 출력
        print("\n[정책 상세 정보]")
        formatted_output = {
            "eligibility_criteria": detail_info.get("eligibility_criteria", "정보 없음"),
            "priority_subjects": detail_info.get("priority_subjects", []),
            "benefits": detail_info.get("benefits", "정보 없음"),
            "required_documents": detail_info.get("required_documents", []),
            "policy_region": detail_info.get("policy_region", "정보 없음"),
            "exclusions": detail_info.get("exclusions", "정보 없음")
        }
        
        print(json.dumps(formatted_output, ensure_ascii=False, indent=2))
        
        # 우대조건 목록이 있다면 별도로 보기 좋게 출력
        if formatted_output["priority_subjects"]:
            print("\n[우대조건 상세]")
            for i, (condition, docs) in enumerate(formatted_output["priority_subjects"], 1):
                print(f"\n{i}. 우대조건: {condition}")
                print(f"   추가서류: {docs}")
    else:
        print("오류:", details.get("message", "상세 정보 추출 실패"))
else:
    print("선택된 정책이 없어 상세 정보를 추출할 수 없습니다.")


[선택된 정책의 상세 정보 추출]
Collection Name: ceongnyeonimcabojeunggeumijajiweonjeongcaeg_body
오류: 상세 정보 추출 중 오류 발생: Expecting value: line 1 column 1 (char 0)
오류: 상세 정보 추출 중 오류 발생: Expecting value: line 1 column 1 (char 0)


In [12]:
# 선택된 정책의 상세 정보 추출
if policy.get("status") == "success":
    collection_name = policy.get("collection_name")
    print("\n[선택된 정책의 상세 정보 추출]")
    print("Collection Name:", collection_name)
    
    try:
        # PolicyDetailTool을 사용하여 상세 정보 추출
        details = detail_tool.run(collection_name)
        
        if details.get("status") == "success":
            detail_info = details.get("details", {})
            
            # 결과가 문자열인 경우 JSON으로 파싱 시도
            if isinstance(detail_info, str):
                try:
                    detail_info = json.loads(detail_info)
                except json.JSONDecodeError:
                    # JSON 파싱 실패 시 기본 구조 생성
                    detail_info = {}
            
            # 결과 출력
            print("\n[정책 상세 정보]")
            formatted_output = {
                "eligibility_criteria": detail_info.get("eligibility_criteria", "정보 없음"),
                "priority_subjects": detail_info.get("priority_subjects", []),
                "benefits": detail_info.get("benefits", "정보 없음"),
                "required_documents": detail_info.get("required_documents", []),
                "policy_region": detail_info.get("policy_region", "정보 없음"),
                "exclusions": detail_info.get("exclusions", "정보 없음")
            }
            
            # 디버깅을 위한 원본 데이터 출력
            print("\n[원본 응답 데이터]")
            print(details.get("details"))
            
            print("\n[정책 상세 정보 (포맷팅)]")
            print(json.dumps(formatted_output, ensure_ascii=False, indent=2))
            
            # 우대조건 목록이 있다면 별도로 보기 좋게 출력
            if formatted_output["priority_subjects"]:
                print("\n[우대조건 상세]")
                for i, (condition, docs) in enumerate(formatted_output["priority_subjects"], 1):
                    print(f"\n{i}. 우대조건: {condition}")
                    print(f"   추가서류: {docs}")
        else:
            print("\n[오류 발생]")
            print("상태:", details.get("status"))
            print("메시지:", details.get("message", "상세 정보 추출 실패"))
            print("\n[전체 응답]")
            print(details)
    except Exception as e:
        print(f"\n[예외 발생]\n{str(e)}")
        print("\n[디버깅 정보]")
        import traceback
        print(traceback.format_exc())
else:
    print("선택된 정책이 없어 상세 정보를 추출할 수 없습니다.")


[선택된 정책의 상세 정보 추출]
Collection Name: ceongnyeonimcabojeunggeumijajiweonjeongcaeg_body

[오류 발생]
상태: error
메시지: 상세 정보 추출 중 오류 발생: Expecting value: line 1 column 1 (char 0)

[전체 응답]
{'status': 'error', 'message': '상세 정보 추출 중 오류 발생: Expecting value: line 1 column 1 (char 0)'}

[오류 발생]
상태: error
메시지: 상세 정보 추출 중 오류 발생: Expecting value: line 1 column 1 (char 0)

[전체 응답]
{'status': 'error', 'message': '상세 정보 추출 중 오류 발생: Expecting value: line 1 column 1 (char 0)'}


In [13]:
# 3) 상세 정보 추출 (검색된 정책이 있을 때만 실행)
if policy.get("status") == "success":
    collection_name = policy.get("collection_name")
    print("\n선택된 정책 collection_name:", collection_name)

    print("상세 정보를 요청합니다 (LLM 호출이 발생할 수 있음) ...")
    details = detail_tool.run(collection_name)
    
    if details.get("status") == "success":
        print("\n[정책 상세 정보 (JSON 형식)]")
        try:
            print(json.dumps(details.get("details"), ensure_ascii=False, indent=2))
        except Exception:
            print(details.get("details"))
    else:
        print("오류 (상세 정보):", details.get("message"))
else:
    print("상세 정보 추출을 위한 정책이 없습니다. 먼저 검색을 수행하세요.")



선택된 정책 collection_name: ceongnyeonimcabojeunggeumijajiweonjeongcaeg_body
상세 정보를 요청합니다 (LLM 호출이 발생할 수 있음) ...
오류 (상세 정보): 상세 정보 추출 중 오류 발생: Expecting value: line 1 column 1 (char 0)
오류 (상세 정보): 상세 정보 추출 중 오류 발생: Expecting value: line 1 column 1 (char 0)


## 실행 방법 및 주의사항

- 이 노트북 셀을 순서대로 실행하세요.
- `PolicyDetailTool.run()`은 내부에서 LLM(예: `gpt-4-mini`)을 호출하므로, 인터넷 연결 및 올바른 OpenAI API 키(또는 환경변수)가 필요합니다.
- `source/vectorstore/header_db`와 `source/vectorstore/body_db`가 프로젝트에 존재해야 하며, suggest_agent_tools.py에 지정된 경로와 일치해야 합니다.
- 만약 `PolicySearchTool`에서 상위 3개를 보고 싶으면, `suggest_agent_tools.py`를 수정하거나 노트북에서 별도 상위-k retriever를 만들어 사용할 수 있습니다.

문제가 있거나 LLM 호출 없이 로컬 데이터만으로 동작 확인을 원하면 알려주세요. (예: body vectorstore에서 직접 텍스트를 읽어 필요한 서류를 키워드로 추출하는 로직 추가)